In [67]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [68]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print(intents)

{u'intents': [{u'patterns': [u'Tere', u'Hei', u'Tsau', u'Tsauki'], u'tag': u'name', u'responses': [u'Tere, mis su nimi on? \U0001f600\U0001f600\U0001f600'], u'context_set': u'name'}, {u'responses': [u'Igastahes, ma olen Mari ja olen 14-aastane. Kui vana sa oled?', u'\U0001f4a9 Kahju! Ma olen Martin, 13. Vana sa?', u'Okei, Ma olen Kadri, kohe saan 12. Kui vana sa praegu oled?'], u'positive': False, u'patterns': [u'Ei \xfctle', u'Ei'], u'tag': u'age_neg', u'context_filter': u'name', u'context_set': u'age'}, {u'responses': [u'R\xf5\xf5m tutvuda, ma olen Mari ja olen 14-aastane. Kui vana sa oled?', u'\U0001f600 Tore! Ma olen Martin, 13. Vana sa?', u'Ma olen Kadri, kohe saan 12. Kui vana sa praegu oled?'], u'positive': True, u'patterns': [u'Olen', u'Minu nimi', u' '], u'tag': u'age_pos', u'context_filter': u'name', u'context_set': u'age'}, {u'patterns': [u'Ei \xfctle', u'Ei'], u'tag': u'home', u'context_filter': u'age', u'responses': [u'Okei! Mul on kodus koer ja me elame perega Viimsis suu

In [69]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(34, 'documents')
(11, 'classes', [u'age_neg', u'age_pos', u'facebook', u'goodbye', u'grade', u'home', u'location', u'name', u'pets', u'subscribe', u'youtube'])
(25, 'unique stemmed words', [u'!', u'4', u'5', u'6', u'7', u'aeg', u'ei', u'head', u'hei', u'kort', u'k\xfcll', u'l\xe4hedal', u'maja', u'minu', u'muj', u'nim', u'n\xe4gemist', u'ol', u'on', u'tallinn', u'tallinna', u'ter', u'tsau', u'tsauk', u'\xfctle'])


In [70]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [71]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 1.07610 | time: 0.017s
| Adam | epoch: 1000 | loss: 1.07610 - acc: 0.6951 -- iter: 32/34
Training Step: 5000  | total loss: 1.07253 | time: 0.019s
| Adam | epoch: 1000 | loss: 1.07253 - acc: 0.6756 -- iter: 34/34
--
INFO:tensorflow:/home/salts/cybersecurity-exhibition/chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [72]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [73]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'age_neg', u'age_pos', u'facebook', u'goodbye', u'grade', u'home', u'location', u'name', u'pets', u'subscribe', u'youtube']


In [74]:
print(model.predict([p]))

[[0.02271738 0.4632715  0.02516184 0.03918431 0.121388   0.02084694
  0.08184329 0.18805069 0.01017467 0.01439557 0.01296587]]


In [75]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )